In [ ]:
# Author: Roberto Doriguzzi-Corin
# Project: Lecture on Intrusion Detection with Deep Learning
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import sys
import time
import glob
import argparse
import tensorflow as tf
import numpy as np
import random as rn
import os
import csv
import h5py
import logging

# Seed Random Numbers
SEED = 1
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)
config = tf.compat.v1.ConfigProto(inter_op_parallelism_threads=1)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel(logging.ERROR)

from itertools import cycle
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.layers import Dense, Activation,  Flatten, Conv2D
from tensorflow.keras.layers import  GlobalMaxPooling2D
from tensorflow.keras.models import Model, Sequential, save_model, load_model, clone_model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, precision_score, accuracy_score, log_loss, confusion_matrix
from sklearn.utils import shuffle

import tensorflow.keras.backend as K
tf.random.set_seed(SEED)
K.set_image_data_format('channels_last')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
#config.log_device_placement = True  # to log device placement (on which device the operation ran)

In [ ]:
# hyperparameters
MAX_EPOCHS=100
LR = 0.01
BATCH_SIZE = 2048
UNITS = 32

In [ ]:
def load_dataset(path):
    filename = glob.glob(path)[0]
    dataset = h5py.File(filename, "r")
    set_x_orig = np.array(dataset["set_x"][:])  # features
    set_y_orig = np.array(dataset["set_y"][:])  # labels

    X = np.reshape(set_x_orig, (set_x_orig.shape[0], set_x_orig.shape[1], set_x_orig.shape[2], 1))
    Y = set_y_orig

    return X, Y

In [ ]:
# MLP model
def MLPModel(model_name, input_shape,units):
    K.clear_session()

    model = Sequential(name=model_name)
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(units, activation='relu', name='fc0'))
    model.add(Dense(units, activation='relu', name='fc1'))
    model.add(Dense(2, activation='softmax', name='fc2'))

    print(model.summary())
    return model

In [ ]:
def compileModel(model,lr):
    optimizer = SGD(learning_rate=lr, momentum=0.9)
    #optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])  # here we specify the loss function

In [ ]:
def trainingEpoch(model, batch_size, X_train,Y_train,X_val,Y_val):
    tt0 = time.time()
    history = model.fit(x=X_train, y=Y_train, validation_data=(X_val, Y_val), epochs=1, batch_size=batch_size, verbose=2, callbacks=[])
    tt1 = time.time()

    accuracy_train = history.history['accuracy'][0]
    loss_train = history.history['loss'][0]
    accuracy_val = history.history['val_accuracy'][0]
    loss_val = history.history['val_loss'][0]
    return tt1-tt0, accuracy_train, loss_train , accuracy_val, loss_val

In [ ]:
X_train, Y_train = load_dataset("../Datasets/DOS2017/*" + '-train.hdf5')
Y_train = to_categorical(Y_train,num_classes=2)
X_val, Y_val = load_dataset("../Datasets/DOS2017/*" + '-val.hdf5')
Y_val = to_categorical(Y_val, num_classes=2)

X_train, Y_train = shuffle(X_train, Y_train, random_state=SEED)
X_val, Y_val = shuffle(X_val, Y_val, random_state=SEED)

In [ ]:
#CNN Model
model = MLPModel("mlp", X_train.shape[1:4], UNITS)
compileModel(model,LR)

In [ ]:
training_fieldnames = ['Samples', 'Time', 'Accuracy(train)', 'Loss(train)', 'Accuracy(val)', 'Loss(val)']
training_file = open('./training-multiclass.csv', 'a', newline='')
training_file.truncate(0)  # clean the file content (as we open the file in append mode)
training_writer = csv.DictWriter(training_file, fieldnames=training_fieldnames)
training_writer.writeheader()
training_file.flush()

In [ ]:
# Training variables
min_loss = float('inf')
max_acc_val = 0
best_model = None

In [ ]:
for epoch in range(MAX_EPOCHS):
    print("Epoch: %d/%s" % (epoch + 1, MAX_EPOCHS))
    ttime, acc_train,loss_train, acc_val, loss_val= trainingEpoch(model, BATCH_SIZE, X_train, Y_train, X_val, Y_val)
    row = {'Samples': Y_train.shape[0], 'Time': '{:10.3f}'.format(ttime), 'Accuracy(train)': acc_train,
           'Loss(train)': loss_train, 'Accuracy(val)': acc_val, 'Loss(val)': loss_val}
    training_writer.writerow(row)

    if acc_val > max_acc_val:
        max_acc_val = acc_val
        best_model_loss_val = loss_val
        best_model = clone_model(model)
        best_model.set_weights(model.get_weights())

In [ ]:
if best_model is not None:
    #Y_pred_val = (best_model.predict(X_val) > 0.5)
    Y_pred_val = np.argmax(best_model.predict(X_val), axis=1)
    #Y_true_val = Y_val.reshape((Y_val.shape[0], 1))
    #f1_score_val = f1_score(Y_true_val, Y_pred_val)
    Y_true_val = np.argmax(Y_val, axis=1)
    f1_score_val = f1_score(Y_true_val, Y_pred_val)

    try:
        best_model.save('model_multiclass.h5')
        print("F1 Score of the saved on the validation set: ", f1_score_val)
    except:
        print("An exception occurred when saving the model!")

In [ ]:
training_file.close()
print("Training log saved in file: ", training_file.name)